# Regressão Logística

## Introdução

Este notebook visa fazer a otimização de **hiperparâmetros**, de estratégia de **normalização**, de **seleção de atributos** e de **redução de dimensionalidade**. A seleção de atributos é realizada por *Recursive Feature Elimination* (**RFE**, ou Eliminação Recursiva de Atributos em português), a otimização testa a ausência ou não dessa estratégia. A redução de dimensionalidade é realizada por *Principal Component Analysis* (**PCA**, ou Análise de Componenetes Principais em português), é testada sua ausência também. Para normalização, são testadas a ausência, ou a normalização **padrão** ou a pelos **mínimos e máximos**. A otimização é feita usando o módulo **optuna**.

### Modelo

O modelo otimizado é uma Regressão Logística do scikit-learn [2]. O argumento 'penalty' é usado na configuração 'elasticnet'. Dessa forma, o único hiperparâmetro otimizado é:

- `l1_ratio`: float entre 0 e 1 que representa a razão entre as regularizações, sendo que 1 (l1) é Lasso e 0 é Ridge (l2);

## Carrega os dados

In [1]:
from pandas impor read_csv

# Dados de treino
X_treino = read_csv('../Dados/dados_tratados/X_treino.csv')
y_treino = read_csv('../Dados/dados_tratados/y_treino.csv').values.ravel()

## Define algumas variáveis úteis

In [2]:
# Parâmetros
SEMENTE_ALEATORIA = 9 # Random state
num_max_atributos = len(X_treino.columns) - 1 # Para RFE

X_treino = X_treino.values

## Definição de funções para o optuna

O código abaixo define a função que cria uma instância do modelo. É nela que o pipeline é montado de acordo com as sugestões definidas pelo optuna.

In [3]:
from sklearn.linear_model import LogisticRegression # algoritmo de aprendizado de máquina
from sklearn.preprocessing import StandardScaler, MinMaxScaler # normalização
from sklearn.feature_selection import RFE # seleção de atributos
from sklearn.decomposition import PCA # redução de dimensionalidade
from sklearn.pipeline import make_pipeline

def cria_instancia_modelo(trial):
    """Cria uma instância do modelo.

    Args:
      trial: objeto tipo Trial do optuna.

    Returns:
      Uma instância do modelo desejado.

    """
    parametros = {
        "penalty": "elasticnet",
        "l1_ratio": trial.suggest_float("razão_regularização", 0, 1), # 0 -> l2, 1 -> l1
        "solver": "saga", # única capaz de resolver elasticnet
        "max_iter": 5000,
        "class_weight": "balanced", # Dados desbalanceados
        "n_jobs": -1,
        "random_state": SEMENTE_ALEATORIA,
    }

    normalizacao = trial.suggest_categorical("normalizacao", ["standard", "minmax"])
    usar_pca = trial.suggest_categorical("PCA", [True, False])
    usar_rfe = trial.suggest_categorical("RFE", [True, False])

    num_atributos = trial.suggest_int("num_atributos", 2, num_max_atributos) # para RFE
    num_dimensoes = trial.suggest_int("num_dimensoes", 2, num_atributos) # para PCA
        
    
    steps = []

    if normalizacao == "standard":
        steps.append(("normalizador", StandardScaler()))
    else:
        steps.append(("normalizador", MinMaxScaler()))

    if usar_rfe:
        steps.append(("rfe", RFE(LogisticRegression(**parametros), n_features_to_select=num_atributos)))

    if usar_pca:
        steps.append(("pca", PCA(n_components=num_dimensoes, random_state=SEMENTE_ALEATORIA)))

    steps.append(("modelo", LogisticRegression(**parametros)))

    modelo = make_pipeline(*[s[1] for s in steps])

    return modelo

O código abaixo define a função que estima o desempenho do modelo de cada trial. Essa é a função que terá valor de retorno maximizado pelo optuna.

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
import numpy as np

f2_score = make_scorer(fbeta_score, beta=2)

def funcao_objetivo(trial, X, y, num_folds):
    """Função objetivo do optuna.

    Faz validação cruzada estratificada com métrica F2.

    """
    modelo = cria_instancia_modelo(trial)

    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring=f2_score,
        cv=num_folds,
    )

    return np.mean(metricas)

## Otimização com optuna

O código abaixo cria ou carrega um 'estudo' a depender da existência dele. O 'estudo' contém informações sobre cada trial. A variável objeto_de_estudo acessa esse 'estudo'. 

In [5]:
from optuna import create_study

NOME_DO_ESTUDO = "Regressao_Logistica_optuna"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-03 21:19:41,637] A new study created in RDB with name: Regressao_Logistica_optuna


O código abaixo faz a otimização com optuna.

In [6]:
# Parâmetros
NUM_FOLDS = 10 # Validação cruzada
NUM_TENTATIVAS = 200 # Quantidade de trials

def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Faz a otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-11-03 21:19:42,578] Trial 0 finished with value: 0.7373487359359844 and parameters: {'razão_regularização': 0.988640698756119, 'normalizacao': 'standard', 'PCA': True, 'RFE': False, 'num_atributos': 4, 'num_dimensoes': 2}. Best is trial 0 with value: 0.7373487359359844.
[I 2025-11-03 21:19:43,125] Trial 1 finished with value: 0.7709377883508247 and parameters: {'razão_regularização': 0.20792770323721643, 'normalizacao': 'standard', 'PCA': True, 'RFE': False, 'num_atributos': 5, 'num_dimensoes': 3}. Best is trial 1 with value: 0.7709377883508247.
[I 2025-11-03 21:19:43,573] Trial 2 finished with value: 0.8595923668227776 and parameters: {'razão_regularização': 0.5268301113325903, 'normalizacao': 'minmax', 'PCA': True, 'RFE': False, 'num_atributos': 5, 'num_dimensoes': 4}. Best is trial 2 with value: 0.8595923668227776.
[I 2025-11-03 21:19:45,515] Trial 3 finished with value: 0.857861205358855 and parameters: {'razão_regularização': 0.6003096685955838, 'normalizacao': 'minmax', '

## DataFrame dos trials

O código abaixo mostra o DataFrame contruído pelo objeto de estudo do optuna contendo todos os parâmetros de cada trial.

In [7]:
df_analysis = objeto_de_estudo.trials_dataframe()

df_analysis

,number,value,datetime_start,datetime_complete,duration,params_PCA,params_RFE,params_normalizacao,params_num_atributos,params_num_dimensoes,params_razão_regularização,state
0,0,0.737349,2025-11-03 21:19:41.670370,2025-11-03 21:19:42.542271,0 days 00:00:00.871901,True,False,standard,4,2,0.988641,COMPLETE
1,1,0.770938,2025-11-03 21:19:42.598742,2025-11-03 21:19:43.097914,0 days 00:00:00.499172,True,False,standard,5,3,0.207928,COMPLETE
2,2,0.859592,2025-11-03 21:19:43.142339,2025-11-03 21:19:43.544635,0 days 00:00:00.402296,True,False,minmax,5,4,0.526830,COMPLETE
3,3,0.857861,2025-11-03 21:19:43.593152,2025-11-03 21:19:45.478827,0 days 00:00:01.885675,False,True,minmax,4,4,0.600310,COMPLETE
4,4,0.818166,2025-11-03 21:19:45.535848,2025-11-03 21:19:47.862296,0 days 00:00:02.326448,True,True,standard,7,3,0.023961,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,0.862489,2025-11-03 21:22:28.756110,2025-11-03 21:22:29.450911,0 days 00:00:00.694801,True,False,minmax,5,4,0.254988,COMPLETE
196,196,0.862489,2025-11-03 21:22:29.556411,2025-11-03 21:22:30.301031,0 days 00:00:00.744620,True,False,minmax,5,4,0.204002,COMPLETE
197,197,0.787975,2025-11-03 21:22:30.392366,2025-11-03 21:22:30.984533,0 days 00:00:00.592167,True,False,standard,5,4,0.153200,COMPLETE
198,198,0.857861,2025-11-03 21:22:31.428628,2025-11-03 21:22:33.267173,0 days 00:00:01.838545,False,True,minmax,5,4,0.230741,COMPLETE


## Melhor modelo

O código abaixo mostra o melhor modelo encontrado pelo optuna e seus parâmetros. Note que os parâmetros 'num_atributos' e 'num_dimensoes' só influenciam caso 'RFE' e 'PCA' sejam True respectivamente, pois são seus respectivos hiperparâmetros.

In [8]:
melhor_trial = objeto_de_estudo.best_trial

print(f"Número do melhor trial: {melhor_trial.number}")
print(f"Parâmetros do melhor trial: {melhor_trial.params}")

Número do melhor trial: 26
Parâmetros do melhor trial: {'razão_regularização': 0.43784276614287854, 'normalizacao': 'minmax', 'PCA': True, 'RFE': False, 'num_atributos': 4, 'num_dimensoes': 4}


Este notebook se encerra aqui. O modelo será usado no notebook chamado 'Resultados_Discussão'.

## Referências

[1] Documentação do optuna. Acesso em: 31/10/2025. Disponível em: https://optuna.readthedocs.io/en/stable/ 

[2] Página da Regressão Logística da documentação do scikit-learn. Acesso em: 31/10/2025. Disponível em: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

[3] Sperat, Walter. *Using optuna with sklearn the right way*. Acesso em 31/10/2025. Disponível em: https://medium.com/@walter_sperat/using-optuna-with-sklearn-the-right-way-part-1-6b4ad0ab2451